###  Importação das bibliotecas

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json

### Procura pela div que possui a classe como quote

In [3]:
def find_the_list_of_quotes(browser):
    while len(browser.find_elements(By.CLASS_NAME, "quote")) == 0:
        time.sleep(2)

### Processa todas as divs que possuem a classe quote

In [4]:
def process_elements(elements, quotes):
    for element in elements:
        quotes.append(get_quote(element))

### Procura pelos elementos das citações  tais como autor, link sobre o autor, frase e as tags 

In [2]:
def get_quote(element):
    quote_dict = {'author': {'name': '', 'url': ''}}
    quote_dict['author']['name'] = element.find_element(By.CLASS_NAME, "author").text
    quote_dict['author']['url'] = element.find_element(By.LINK_TEXT,'(about)').get_attribute('href')
    quote_dict['quote'] = element.find_element(By.CLASS_NAME, "text").text.replace("“", "").replace("”", "")
    element_tags = element.find_element(By.CLASS_NAME, "tags")
    element_tag = element_tags.find_elements(By.CLASS_NAME, "tag")
    quote_dict['author']['name'].encode("utf-8")
    quote_dict['quote'].encode("utf-8")
    tags = []
    for tag in element_tag:
        tags.append(tag.text)
    quote_dict['tags'] = tags
    return quote_dict

### Pega o XPATH do botão Next. Na primeira página como não tem o botão Previous o XPATH é diferente das demais páginas

In [5]:
def get_xpath_buttom(page):
    if page == 1:
        return "/html/body/div/div[2]/div[1]/nav/ul/li/a"
    else:
        return "/html/body/div/div[2]/div[1]/nav/ul/li[2]/a"

### Procura pelo botão Next. Caso não ache na primeira vez, será verificado novamente depois de 5 secondos. 

In [6]:
def find_buttom(xpath_buttom, browser):
    if len(browser.find_elements(By.XPATH, xpath_buttom)) == 0:
        time.sleep(5)
    return True if len(browser.find_elements(By.XPATH, xpath_buttom)) == 1 else False

### Escrever todas as citações detectadas em um arquivo JSON(JavaScript Object Notation)

In [7]:
def write_in_json_file(quotes):
    with open("./all_quotes.json", "w") as outfile:
        json.dump(quotes, outfile, indent=5)

### Função Main

In [8]:
def main():
    browser = webdriver.Chrome()
    browser.get("http://quotes.toscrape.com")
    
    page = 1
    quotes = []
    
    while True:
        find_the_list_of_quotes(browser)
        elements = browser.find_elements(By.CLASS_NAME, "quote")
        process_elements(elements, quotes)
        xpath_buttom = get_xpath_buttom(page)
        if find_buttom(xpath_buttom,browser):
            browser.find_element(By.XPATH, xpath_buttom).click()
            page += 1
            time.sleep(5)
        else:
            break
    
    write_in_json_file(quotes)

### Execução do código

In [9]:
main()